In [1]:
from sympy import *
import numpy as np
from scipy.special import airy
from scipy.integrate import quad
import matplotlib.pyplot as plt
from collections.abc import Iterable
import ipywidgets as widgets

In [2]:
t = symbols('t', positive=True)

w = Function('w')

def dsum_Barnett(n, t):
    """
    Sum of the first (n+1) terms in the Barnett series, which approximates 
    u(t) following the sequence of substitutions below:
    x'' + w(t)^2x = 0,
    let x = e^z,
    let z' = u.
    """
    if n == 0:
        return 1j*w(t)
    else:
        return Rational(1,2)*(-diff(dsum_Barnett(Integer(n-1), t), t)/dsum_Barnett(Integer(n-1), t) +\
                    dsum_Barnett(Integer(n-1), t) - w(t)**Integer(2)/dsum_Barnett(Integer(n-1), t))
    
def dS_WKB(n, t):
    """
    Derivative of the (n+1)th term in the WKB series. 
    """
    if n == 0:
        return 1j*w(t)
    else:
        sums = 0.0
        for i in range(1, n):
            sums += dS_WKB(i,t)*dS_WKB(n-i, t)
        return -1/(2*dS_WKB(0, t))*(diff(dS_WKB(n-1, t), t) + sums)
    
def sum_dS_WKB(n, t):
    """
    Sum of the derivatives of the first (n+1) terms in the WKB series.
    """
    sums = 0.0
    for i in range(n+1):
        sums += dS_WKB(i, t)
    return sums

def eval_Barnett(n, ts, x0, dx0):
    """
    Evaluates the Barnett series approximation to x(t) at a sequence of
    points ts, starting from initial condititions x(t[0]) = x0,
    x'(t[0]) = dx0.
    """
    t0 = ts[0]
    dBarnett = dsum_Barnett(n, t)
    dBarnett_re = re(dBarnett)
    dBarnett_im = im(dBarnett)
    #Barnett = integrate(dBarnett, t)
    #Barnett_lambda = lambdify(t, Barnett, 'numpy')
    dBarnett_lambda = lambdify(t, dBarnett, 'numpy')
    dBarnett_lambda_re = lambdify(t, dBarnett_re, 'numpy')
    dBarnett_lambda_im = lambdify(t, dBarnett_im, 'numpy')
    Barnett_lambda = lambda us: np.array([quad(dBarnett_lambda_re, t0, u)[0] +\
                                          1j*quad(dBarnett_lambda_im, t0, u)[0] for u in us])\
                                          if isinstance(us, Iterable) else quad(dBarnett_lambda_re, t0, us)[0] +\
                                          1j*quad(dBarnett_lambda_im, t0, us)[0]
                                    
    fp = np.exp(Barnett_lambda(ts) - Barnett_lambda(t0))
    fm = np.conj(fp)
    dfp0 =  dBarnett_lambda(t0)
    dfm0 = np.conj(dfp0)
    Ap = (dx0 - x0*dfm0)/(dfp0 - dfm0)
    Am = (dx0 - x0*dfp0)/(dfm0 - dfp0)
    result = Ap*fp + Am*fm
    return result
    
def eval_WKB(n, ts, x0, dx0):
    """
    Evaluates the WKB approximation to x(t) at a sequence of points ts,
    starting from initial condititions x(t[0]) = x0, x'(t[0]) = dx0.
    """
    t0 = ts[0]
    dWKB = sum_dS_WKB(n, t)
    WKB = integrate(dWKB, t)
    WKB_lambda = lambdify(t, WKB, 'numpy')
    dWKB_lambda = lambdify(t, dWKB, 'numpy')
    fp = np.exp(WKB_lambda(ts) - WKB_lambda(t0))
    fm = np.conj(fp)
    dfp0 =  dWKB_lambda(t0)
    dfm0 = np.conj(dfp0)
    Ap = (dx0 - x0*dfm0)/(dfp0 - dfm0)
    Am = (dx0 - x0*dfp0)/(dfm0 - dfp0)
    result = Ap*fp + Am*fm
    return result    

In [9]:
dsum_Barnett(0, t)

1.0*I*w(t)

In [6]:
dsum_Barnett(1, t)

I*w(t) - 0.5*Derivative(w(t), t)/w(t)

In [16]:
Barnett2 = simplify(dsum_Barnett(2, t))
Barnett2

(2.0*w(t)**4 + 2.0*I*w(t)**2*Derivative(w(t), t) - 0.5*w(t)*Derivative(w(t), (t, 2)) + 0.25*Derivative(w(t), t)**2)/((-2.0*I*w(t)**2 + 1.0*Derivative(w(t), t))*w(t))

In [17]:
WKB2 = simplify(sum_dS_WKB(2, t))
WKB2

(I*(-0.25*w(t)*Derivative(w(t), (t, 2)) + 0.375*Derivative(w(t), t)**2) + 1.0*I*w(t)**4 - 0.5*w(t)**2*Derivative(w(t), t))/w(t)**3

In [15]:
simplify(Barnett2 - WKB2)

I*(0.25*w(t)*Derivative(w(t), (t, 2)) - 0.375*Derivative(w(t), t)**2)*Derivative(w(t), t)/((-2.0*I*w(t)**2 + 1.0*Derivative(w(t), t))*w(t)**3)

In [18]:
Barnett3 = simplify(dsum_Barnett(3, t))
Barnett3

(-128.0*w(t)**8 - 256.0*I*w(t)**6*Derivative(w(t), t) + 64.0*w(t)**5*Derivative(w(t), (t, 2)) + 96.0*w(t)**4*Derivative(w(t), t)**2 + 16.0*I*w(t)**4*Derivative(w(t), (t, 3)) - 16.0*I*w(t)**3*Derivative(w(t), t)*Derivative(w(t), (t, 2)) + 40.0*I*w(t)**2*Derivative(w(t), t)**3 - 8.0*w(t)**2*Derivative(w(t), t)*Derivative(w(t), (t, 3)) + 4.0*w(t)**2*Derivative(w(t), (t, 2))**2 + 8.0*w(t)*Derivative(w(t), t)**2*Derivative(w(t), (t, 2)) - 5.0*Derivative(w(t), t)**4)/((128.0*I*w(t)**6 - 192.0*w(t)**4*Derivative(w(t), t) - 32.0*I*w(t)**3*Derivative(w(t), (t, 2)) - 48.0*I*w(t)**2*Derivative(w(t), t)**2 + 16.0*w(t)*Derivative(w(t), t)*Derivative(w(t), (t, 2)) - 8.0*Derivative(w(t), t)**3)*w(t))

In [19]:
WKB3 = simplify(sum_dS_WKB(3, t))
WKB3

1.0*I*w(t) - 0.5*Derivative(w(t), t)/w(t) - 0.25*I*Derivative(w(t), (t, 2))/w(t)**2 + 0.375*I*Derivative(w(t), t)**2/w(t)**3 + 0.125*Derivative(w(t), (t, 3))/w(t)**3 - 0.75*Derivative(w(t), t)*Derivative(w(t), (t, 2))/w(t)**4 + 0.75*Derivative(w(t), t)**3/w(t)**5

In [20]:
simplify(Barnett3 - WKB3)

(16.0*w(t)**6*Derivative(w(t), t)*Derivative(w(t), (t, 3)) + 12.0*w(t)**6*Derivative(w(t), (t, 2))**2 - 128.0*w(t)**5*Derivative(w(t), t)**2*Derivative(w(t), (t, 2)) + 4.0*I*w(t)**5*Derivative(w(t), (t, 2))*Derivative(w(t), (t, 3)) + 117.0*w(t)**4*Derivative(w(t), t)**4 + 6.0*I*w(t)**4*Derivative(w(t), t)**2*Derivative(w(t), (t, 3)) - 20.0*I*w(t)**4*Derivative(w(t), t)*Derivative(w(t), (t, 2))**2 - 20.0*I*w(t)**3*Derivative(w(t), t)**3*Derivative(w(t), (t, 2)) - 2.0*w(t)**3*Derivative(w(t), t)*Derivative(w(t), (t, 2))*Derivative(w(t), (t, 3)) + 39.0*I*w(t)**2*Derivative(w(t), t)**5 + 1.0*w(t)**2*Derivative(w(t), t)**3*Derivative(w(t), (t, 3)) + 12.0*w(t)**2*Derivative(w(t), t)**2*Derivative(w(t), (t, 2))**2 - 18.0*w(t)*Derivative(w(t), t)**4*Derivative(w(t), (t, 2)) + 6.0*Derivative(w(t), t)**6)/((128.0*I*w(t)**6 - 192.0*w(t)**4*Derivative(w(t), t) - 32.0*I*w(t)**3*Derivative(w(t), (t, 2)) - 48.0*I*w(t)**2*Derivative(w(t), t)**2 + 16.0*w(t)*Derivative(w(t), t)*Derivative(w(t), (t, 2)) 